# 實驗說明

## 歷史幣價資料取得

- [幣安API](https://binance-docs.github.io/apidocs/futures/cn/#185368440e)

## 幣種

- BTC
- ETH
- BNB
- SOL
- BUSD

## 資料切分

- 訓練時間段 : 2020/10/17 ~ 2021/10/17
- 測試時間段 : 2021/10/18 ~ 2023/10/25

## 避險機制

#### 邏輯
- 比特幣做為加密貨幣的元老，其走勢往往能牽動其他的加密貨幣，例如比特幣大漲時會帶動其他小幣可能一起漲，甚至漲得更多，但比特幣大跌時同樣也會讓其他小幣一起大跌，如同加密貨幣的大盤

#### 原理
- 利用比特幣的大盤特性計算一種動能指標來判斷當前市場是樂觀情緒(利於做多)還是悲觀情緒(不利於做多)

- 在比特幣動能<0時判定為不利於做多的市場，將所有幣種賣出並轉換成穩定幣以度過進一步下跌風險
- 在比特幣動能>0後判定為利於做多的市場，再將穩定幣換成各個幣種以吃到瞬間漲幅

## 特徵
- [眾多技術指標](https://github.com/bukosabino/ta)
- [SuperTrend](https://tw.tradingview.com/scripts/supertrend/) 指標
- [Catch22](https://github.com/DynamicsAndNeuralSystems/pycatch22) 時間序列特徵
- 過去平均跌幅
- 微軟[QLib](https://github.com/microsoft/qlib) 開高低收特徵

## 模型
- 強化學習演算法 [A2C](https://github.com/openai/baselines/tree/master/baselines/a2c)

## 回測結果分析

- 比較基準為100%持有比特幣

#### 重要的衡量指標
- Alpha : 相對於大盤(比特幣)的超額報酬，愈高愈好，表示無考慮風險的賺錢能力
- Sharp Ratio : 承受一單位的風險可以獲得多少單位的報酬，愈高愈好，表示有考慮風險的賺錢能力
- CAGR : 年複合增長率，平均一年成長了多少%，愈高愈好
- Calmar Ratio : CAGR/Max Drawdown，愈高愈好，表示考慮最大風險的賺錢能力
- Mean Drawdown : 平均跌幅，愈低愈好，表示一期間內平均跌了多少%，表示抗跌能力
- Max Drawdown : 最大跌幅，愈低愈好，表示曾經跌了多少%
- Prob of Losing Money : 一期間內虧錢期間佔了多少%，愈低愈好，表示虧錢期間的長短

#### 從報酬的角度來看
- 2021年末的短暫牛市報酬大於單純持有比特幣
- 2023年比特幣從低點反彈並有幾段牛市，因此模型表現不如單純持有比特幣
- 2021/10/18~2023/10/25 獲利表現 44% 勝單純持有比特幣的-43%

#### 從風險的角度來看
- 2022年熊市可以透過避險機制避開數次大跌，跌幅遠低於單純持有比特幣
- 最大回撤 -41% 勝單純持有比特幣的76%
- 平均回撤 -17% 勝單純持有比特幣的55%

## 結論

- 長期持有加密貨幣是一件高風險高報酬的事情，例如單純持有比特幣，雖然上漲時漲幅高，但下跌時跌幅也很高，並且很依靠進場的位置，是否屬於低點。本研究使用A2C強化學習模型並配合避險機制風控一個長期持有加密貨幣的投資組合，實驗結果表明可以在報酬與風險中間達到一個不錯的平衡，為投資人提供一個除了單純持有比特幣以外的新選擇。

# 連結雲端硬碟

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 安裝套件

In [6]:
!pip install stable-baselines3==1.2.0 > log.txt
!pip install optuna > log.txt
!pip install ta==0.9.0 > log.txt
!pip install git+https://github.com/DynamicsAndNeuralSystems/pycatch22.git
# Install ta-lib
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib > log.txt

  Cloning https://github.com/DynamicsAndNeuralSystems/pycatch22.git to /tmp/pip-req-build-sxjgwljw
  Running command git clone --filter=blob:none --quiet https://github.com/DynamicsAndNeuralSystems/pycatch22.git /tmp/pip-req-build-sxjgwljw
  Resolved https://github.com/DynamicsAndNeuralSystems/pycatch22.git to commit e3c35713dcf35fb97a6eae0271fb417d2db0ec1f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
(Reading database ... 121944 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/l

*斜體文字*# 設定參數

In [ ]:
# 雲端硬碟檔案存放路徑
FOLDER = '/content/drive/MyDrive/CryptoPortfolioRL2023/'

# 設定回測比較基準，如比特幣
BENCHMARK = 'BTC/USDT'

# 一開始的投資金額
INITIAL_AMOUNT = 10000

# 訓練回合數
N_TIMESTEPS = 50000

# 是否使用避險機制 : [True] [False]
HEDGING = [True]

# 模型演算法選擇 : 'a2c' 'ppo'
AGENT_NAME = 'ppo'

# 模型超參數
A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.002}
PPO_PARAMS = {"n_steps": 2048,"ent_coef": 0.005,"learning_rate": 0.001,"batch_size": 128}

MODEL_KWARGS = {"a2c": A2C_PARAMS, "ppo": PPO_PARAMS}

# 訓練與測試時間段
TRAIN_START_DATE = '2020-10-17'
TRAIN_END_DATE = '2021-10-17'
TRADE_START_DATE = '2021-10-18'
TRADE_END_DATE = '2023-10-25'

# 投資組合中的加密貨幣
CRYPTO_LIST = ['BTC', 'ETH', 'BNB', 'SOL']

# 加入所有特徵 (預設)
FEATURE_LIST = ['86ta', 'dd', 'st', 'catch22', 'qlib']

if HEDGING[0]:
    CRYPTO_LIST.append('BUSD')
    FEATURE_LIST.append('hedging')

    index_busd = sorted(CRYPTO_LIST).index('BUSD')
    HEDGING.append(index_busd)

CRYPTO_LIST = [ticker + '/USDT' for ticker in CRYPTO_LIST]

[連結文字](https://)# 資料、環境建置、模型訓練

In [ ]:
from data import *

data_kwargs = {
    "folder": FOLDER,
    "crypto_list": CRYPTO_LIST,
    "feature_list": FEATURE_LIST,
    "train_start_date": TRAIN_START_DATE,
    "train_end_date": TRAIN_END_DATE,
    "trade_start_date": TRADE_START_DATE,
    "trade_end_date": TRADE_END_DATE
}
# 讀取、計算特徵、並切分成訓練、測試資料
train, trade, tech_indicator_list = prepare_data(**data_kwargs)

from training import *

env_kwargs = {
    "transaction_cost_pct": 0.001,
    "reward_scaling": 1e-4,
    "initial_amount": INITIAL_AMOUNT,
    "tech_indicator_list": tech_indicator_list,
    "hedging": HEDGING,
    "folder": FOLDER
}
# 建立強化學習的環境
e_train_gym, e_trade_gym, env_train, crypto_dimension, state_space = building_environment(env_kwargs, train, trade)

train_kwargs = {
    "train": train,
    "trade": trade,
    "e_train_gym": e_train_gym,
    "e_trade_gym": e_trade_gym,
    "env_train": env_train,
    "agent_name": AGENT_NAME,
    "total_timesteps":N_TIMESTEPS,
    "MODEL_KWARGS": MODEL_KWARGS,
    "folder": FOLDER
}
# 訓練模型，並讓模型預測測試時間段的結果
df_daily_return, df_actions, trained_agent = train_test_agent(**train_kwargs)

Crypto Dimension: 5, State Space: 5
reset at 2020-10-17T08:00:00.000000000
reset at 2021-10-18T08:00:00.000000000
{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.001, 'batch_size': 128}
Using cpu device
reset at 2020-10-17T08:00:00.000000000
Logging to ./ppo/ppo_1
begin_total_asset:10000
end_total_asset:150433.72796306154
Sharpe:  0.22568150771049467
reset at 2020-10-17T08:00:00.000000000
begin_total_asset:10000
end_total_asset:136709.96555826176
Sharpe:  0.2195797051393616
reset at 2020-10-17T08:00:00.000000000
begin_total_asset:10000
end_total_asset:110925.00842095552
Sharpe:  0.20378156200174194
reset at 2020-10-17T08:00:00.000000000
begin_total_asset:10000
end_total_asset:149932.33899116144
Sharpe:  0.22560005393735277
reset at 2020-10-17T08:00:00.000000000
begin_total_asset:10000
end_total_asset:129418.69240836619
Sharpe:  0.20869640089891217
reset at 2020-10-17T08:00:00.000000000
-----------------------------
| time/              |      |
|    fps             | 32   |
|  

In [ ]:
from backtesting import *
# 讀取模型歷史每日投資組合權重和報酬率的檔案
df_daily_return = pd.read_csv(f'{FOLDER}df_daily_return_{AGENT_NAME}_crypto.csv', index_col=0, parse_dates=True)
df_actions = pd.read_csv(f'{FOLDER}df_actions_{AGENT_NAME}_crypto.csv', index_col=0, parse_dates=True)
# 讀取回測比較基準的資料
bm, close_data = prepare_backtesting_data(df_daily_return, df_actions, BENCHMARK, FOLDER)

# 限定資料頻率為8小時 (原本是頻率4小時)
bm = bm[bm.index.hour==8]
close_data = close_data[close_data.index.hour==8]

CASH = 10000
NAME = 'DRL Agent'
# 回測分析
backtest_analytics(df_daily_return, bm, df_actions, close_data, cash=CASH, NAME=NAME, BM_NAME=BENCHMARK)

# 回測結果